# 保持与水肿识别的ID一致

In [13]:
import pandas as pd
import os

df = pd.read_csv('/data1/lzf/301MRI/u-disk/res/classify/classify_8/final_classify_res_resnet18_fold_3.csv')
print(df.head())

          file_name  split         0         1  pred  label
0  236016_3_ilu.npy  train  0.953598  0.046402     0    0.0
1  236016_3_iru.npy  train  0.955908  0.044092     0    0.0
2  236016_3_slu.npy  train  0.740474  0.259526     0    0.0
3  236016_3_sru.npy  train  0.897397  0.102603     0    0.0
4  236016_4_ild.npy  train  0.811941  0.188059     0    0.0


In [14]:
train_list = df[df['split'] == 'train']["file_name"].tolist()
train_list = [item.split("_")[0] for item in train_list]
train_list = list(set(train_list))
print(len(train_list))

valid_list = df[df['split'] == 'valid']["file_name"].tolist()
valid_list = [item.split("_")[0] for item in valid_list]
valid_list = list(set(valid_list))
print(len(valid_list))

test_list = df[df['split'] == 'test']["file_name"].tolist()
test_list = [item.split("_")[0] for item in test_list]
test_list = list(set(test_list))
print(len(test_list))

442
110
139


In [15]:
# 把list写入txt
with open("/data1/lzf/301agent/data/train_list0411.txt", "w") as f:
    for item in train_list:
        f.write(item + "\n")
with open("/data1/lzf/301agent/data/valid_list0411.txt", "w") as f:
    for item in valid_list:
        f.write(item + "\n")
with open("/data1/lzf/301agent/data/test_list0411.txt", "w") as f:
    for item in test_list:
        f.write(item + "\n")

In [16]:
df2 = pd.read_csv("/data1/lzf/301agent/data/agent_sheet_all_0409.csv")
patient_id = df2["patient_id"].tolist()
print(len(patient_id))

545


In [17]:
train_list = list(set(train_list) & set(patient_id))
valid_list = list(set(valid_list) & set(patient_id))
test_list = list(set(test_list) & set(patient_id))
print(len(train_list))
print(len(valid_list))
print(len(test_list))

359
85
101


In [18]:
test_list = valid_list + test_list
print(len(train_list))
print(len(test_list))

359
186


In [19]:
with open("/data1/lzf/301agent/code_0411/train_list.txt", "w") as f:
    for item in train_list:
        f.write(item + "\n")
with open("/data1/lzf/301agent/code_0411/test_list.txt", "w") as f:
    for item in test_list:
        f.write(item + "\n")


# 专病大模型

In [4]:
import requests
import json
# curl http://121.40.225.8:8003/v1/chat/completions -H "Content-Type: application/json" -d '{
# "model": "axspa-3xx",
# "messages": [
#     {"role": "user", "content": "你可以做什么"}
# ],
# "do_sample": false,
# "stream": false
# }'
class ChatAPIClient:
    def __init__(self, base_url="http://121.40.225.8:8003/v1/chat/completions", model="axspa-3xx"):
        self.base_url = base_url
        self.model = model
        self.headers = {
            "Content-Type": "application/json"
        }
        self.history = []

    def generate_res(self, user_input):
        # 添加用户消息到历史记录
        self.history.append({"role": "user", "content": user_input})

        payload = {
            "model": self.model,
            "messages": self.history,
            "do_sample": False,
            "stream": False  # 非流式响应
        }

        response = requests.post(self.base_url, headers=self.headers, data=json.dumps(payload))

        if response.status_code == 200:
            data = response.json()
            # 假设返回格式为 OpenAI 接口兼容格式
            reply = data["choices"][0]["message"]["content"]
            # 添加 assistant 回复到历史
            self.history.append({"role": "assistant", "content": reply})
            return reply
        else:
            raise Exception(f"API调用失败，状态码: {response.status_code}, 信息: {response.text}")

    def reset_hist(self):
        """清空历史记录"""
        self.history = []

# 使用示例
# if __name__ == "__main__":
#     client = ChatAPIClient()
#     client.generate_res("你可以做什么")
#     client.generate_res("你能写首诗吗？")
#     client.reset_hist()  # 清空历史对话
#     client.generate_res("我们重新开始吧")


In [7]:
client = ChatAPIClient()

In [8]:
res1 = client.generate_res("介绍一下你自己")
print(res1)
res2 = client.generate_res("什么是axSpA")
print(res2)
res3 = client.generate_res("有哪些治疗方式")
print(res3)
client.reset_hist()  # 清空历史对话
res4 = client.generate_res("有哪些治疗方式")
print(res4)

当然，我是Qwen，阿里云研发的超大规模语言模型。我拥有多样的技能，可以和你聊天，提供信息，写故事、写公文、写代码、写邮件、写剧本等等，还能表达观点，总结信息和做数学运算等。我对中文和英文都有良好的理解，使用时请尽量用标准的语言，避免方言和俚语，这样交流起来会更顺畅。
axSpA，即银屑病关节炎（Psoriatic Arthritis，PsA）的另一种表达方式，是一种慢性炎症性关节炎，与银屑病（俗称牛皮癣）有关。银屑病是一种常见的自身免疫性疾病，主要表现为皮肤和指甲的损害。大约30%的银屑病患者可能会发展成银屑病关节炎，这种关节炎不仅会影响关节，还可能累及皮肤、指甲、眼睛等部位。

银屑病关节炎的症状包括但不限于：关节疼痛、肿胀、僵硬，特别是早晨起床时或长时间休息后；也可能出现指甲变形、脱落或出现凹陷；皮肤上可能出现红色、鳞状的斑块；眼睛红痛等。

诊断银屑病关节炎通常需要结合患者的病史、体格检查以及相关的实验室检查和影像学检查。如果怀疑患有银屑病关节炎，建议及时就医，由专业医生进行评估和治疗。
银屑病关节炎（PsA）的治疗目标是减轻症状、控制炎症、防止关节损伤、改善功能和生活质量。治疗方法主要包括药物治疗、物理治疗、手术治疗和其他辅助治疗方法。以下是一些常见的治疗方式：

1. **非甾体抗炎药（NSAIDs）**：用于缓解疼痛和减少炎症，如布洛芬、萘普生等。

2. **免疫抑制剂**：如甲氨蝶呤、环孢素等，用于抑制免疫系统，减少炎症。

3. **生物制剂**：如肿瘤坏死因子抑制剂（TNF抑制剂）、白细胞介素抑制剂（如IL-17和IL-23抑制剂）等，针对特定的免疫途径，效果较好但费用较高。

4. **Janus激酶抑制剂（JAK抑制剂）**：如托法替布等，通过抑制信号传导途径来减少炎症。

5. **物理治疗**：通过运动和拉伸来保持关节活动度，增强肌肉力量，减轻疼痛。

6. **手术治疗**：对于严重的关节损伤，可能需要进行关节置换手术或其他手术治疗。

7. **生活方式调整**：包括健康饮食、戒烟、适度运动等，有助于改善症状和整体健康状况。

8. **心理支持和教育**：提供心理支持和教育，帮助患者更好地应对疾病。

治疗方案应根据患者的具体情况由医生制定，包括疾病类型、严重程度、并发症等因素。建议在医生的指导下选择合适的治疗方法，以获得最佳疗效

In [9]:
print(res2)

axSpA，即银屑病关节炎（Psoriatic Arthritis，PsA）的另一种表达方式，是一种慢性炎症性关节炎，与银屑病（俗称牛皮癣）有关。银屑病是一种常见的自身免疫性疾病，主要表现为皮肤和指甲的损害。大约30%的银屑病患者可能会发展成银屑病关节炎，这种关节炎不仅会影响关节，还可能累及皮肤、指甲、眼睛等部位。

银屑病关节炎的症状包括但不限于：关节疼痛、肿胀、僵硬，特别是早晨起床时或长时间休息后；也可能出现指甲变形、脱落或出现凹陷；皮肤上可能出现红色、鳞状的斑块；眼睛红痛等。

诊断银屑病关节炎通常需要结合患者的病史、体格检查以及相关的实验室检查和影像学检查。如果怀疑患有银屑病关节炎，建议及时就医，由专业医生进行评估和治疗。


# sparcc_df

In [1]:
import pandas as pd

sparcc_df = pd.read_csv("/data1/lzf/301agent/data/sparcc_label_df.csv")
sparcc_df.head(3)

,id,iru,sru,ird,srd,rdeep,rinten,slu,ilu,sld,ild,ldeep,linten
0,236016_3,0,0,0,0,0,0,0,0,0,0,0,0
1,236016_4,0,0,0,0,0,0,0,0,0,0,0,0
2,236016_5,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:



def get_sparcc_text_by_id(patient_id, sparcc_df):
    patient_rows = sparcc_df[sparcc_df["id"].str.contains(patient_id)]
    sparcc_total = patient_rows[["iru", "sru", "ird", "srd", "rdeep", "rinten", "slu", "ilu", "sld", "ild", "ldeep", "linten"]].sum(axis=1).sum()
    patient_rows = patient_rows.reset_index(drop=True)
    # 转化为字符串
    sparcc_text = patient_rows.to_string(index=False)
    text = "SPARCC评分表格,id列表示患者id和对应的层数：\n" + sparcc_text + "\nSPARCC总分：" + str(sparcc_total)
    return text

In [12]:
patient_id = "Y6759066"
print(get_sparcc_text_by_id(patient_id, sparcc_df))

SPARCC评分表格,id列表示患者id和对应的层数：
        id  iru  sru  ird  srd  rdeep  rinten  slu  ilu  sld  ild  ldeep  linten
Y6759066_4    0    0    0    0      0       0    1    1    0    0      0       0
Y6759066_5    0    0    0    0      0       0    1    1    0    0      0       0
Y6759066_6    0    0    0    0      0       0    1    1    0    0      0       0
Y6759066_7    0    1    0    1      0       0    1    1    1    1      0       0
Y6759066_8    0    0    0    1      0       0    1    1    1    1      1       0
Y6759066_9    0    0    0    0      0       0    1    1    1    1      0       0
SPARCC总分：22
